
Projet de Combinatoire et Énumération
====== 

[Sujet du projet](https://github.com/hivert/CombiFIIL/raw/master/SujetProjet/projet.pdf)

L'objectif de ce projet est d'implémenter un comptage d'objets combinatoire décrits par une grammaire quelconque, on utilisera pour ce faire les méthodes générales de produit cartésien et d'union disjointe vues en cours.



## Questions de cours

### Question 1

Pour les grammaires des arbres et des mots de Fibonnacci, donner dans un tableau pour
n = 0, . . . , 10 les réponses attendue pour la méthode count pour les 8 non terminaux
des mots de Fibonnacci et les 3 non terminaux des arbres binaires

_______________

|         |  0|  1|  2|  3|  4|  5|  6|  7|  8|  9| 10|
|:---     |---|---|---|---|---|---|---|---|---|---|---|
| Fib     |  1 | 2  | 3  | 5  |  8 | 13  | 21  | 34  | 55  |  89 | 144  |
| Cas1    |  0 | 2  | 3  | 5  | 8  | 13  | 21  | 34  | 55  | 89  | 144  |
| Cas2    |  0 | 1  | 1  | 2  | 3  | 5  | 8  | 13  | 21  | 34  | 55  |
| Vide    | 1  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  |
| CasAu   | 0  | 1  | 2  | 3  | 5  | 8  | 13  | 21  | 34  | 55  | 89  |
| AtomA   |  0 | 1  | 0  | 0  | 0  | 0  | 0  | 0  |  0 | 0  | 0  |
| AtomB   | 0  | 1  | 0  | 0  | 0  | 0  | 0  | 0  |  0  | 0 | 0  |
| CasBAu  | 0  | 0  | 1  | 2  | 3  | 5  | 8  | 13  | 21  | 34  | 55  |

|         |  0|  1|  2|  3|  4|  5|  6|  7|  8|  9| 10|
|:---     |---|---|---|---|---|---|---|---|---|---|---|
| Tree    |  0 | 1  | 1  | 2  |  5 | 14  | 42  | 132  | 429  |  1430 | 4862  |
| Node    | 0  | 0  | 1  | 2  | 5  | 14  | 42  |  132 | 429  | 1430  | 4862  |
| Leaf    | 0  | 1  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  |


### Question 2

Donner la grammaire de tous les mots sur l’alphabet A,B.

____________


### Question 3

Donner la grammaire des mots de Dyck, c’est-à-dire les mots sur l’alphabet {(, )} et
qui sont correctement parenthésés.

____________


### Question 4

Donner la grammaire de mots sur l’alphabet A,B qui n’ont pas trois lettres consécuti-
vement égales.

____________

### Question 5

Donner la grammaire des palindromes sur l’alphabet A, B, même question sur l’alphabet
A,B,C.

_____________


### Question 6

Donner la grammaire des mots sur l’alphabet A,B qui contiennent autant de lettres A
que de lettres B.

______________


### Question 7

Écrire une fonction qui vérifie qu’une grammaire est correcte, c’est-à-dire que chaque
non-terminal apparaissant dans une règle est bien défini par la grammaire.

____________

## Implémentation des classes de grammaire

Nous commençons par implémenter les différentes classes qui modéliseront la grammaire en Python, on suivra la hierarchie suivante:

![](./combi_class_hierarchy.png)


- __AbstractRule__: Représente une abstraction des règles de la grammaire, en particulier elle implémentera un dictionnaire qui fera référence à la grammaire toute entière ainsi qu'une fonction __set_grammar__ qui permettra de modifier la grammaire (?)
- __ConstructorRule__: Règles de constructions qui répresentent les symboles non-terminaux et qui sont dans notre cas __UnionRule__ et __ProductRule__
    - ProductRule : Cette règle représente le produit cartésien de deux règles, elle hérite de la classe ConstructorRule
    - UnionRule : Il s'agit de l'union disjointe de deux règles, elle hérite également de la classe ConstructorRule
- __ConstantRule__: Règles constantes représentant les symboles terminaux
    - EpsilonRule: Règle représentant le symbole $\epsilon$
    - SingletonRule: Règle représentant un symbole terminal quelconque

In [ ]:
from abc import ABCMeta, abstractmethod

class AbstractRule(metaclass=ABCMeta):

    def __init__(self, constant):
        self._grammar = {}
        self._constant = constant

    def _set_grammar(self, gram):
        self._grammar = gram
        
    def isConstant():
        return self._constant
    
    @abstractmethod
    def count(self, n):
        """count the numbers of objects of weight n"""
    
    @abstractmethod
    def list(self, n):
        """Enumerate objects of weight n that can be generated by the grammar and return them as a list"""
        
    @abstractmethod
    def unrank(self, n, rank):
        """Return object of size n and rank rank"""

class ConstantRule(AbstractRule,metaclass=ABCMeta):
    def __init__(self, pyobj, valuation):
        super().__init__(True)
        self._object = pyobj
        self.__valuation = valuation
        
    @property
    def valuation(self):
        return self.__valuation
    
    def unrank(self, n, rank):
        return self._object


class ConstructorRule(AbstractRule,metaclass=ABCMeta):
    def __init__(self, value):
        super().__init__(False)
        self._valuation = float('inf')
        self._parameters = value

    @property
    def valuation(self):
        return self._valuation
    
    @property
    def rules(self):
        return tuple(map(lambda p : self._grammar[p], self._parameters))

    @abstractmethod
    def _calc_valuation(self):
        """update valuation"""

    def _update_valuation(self):
        self._grammar[self._parameters[0]]._update_valuation()


## Implémentation des classes héritières et algorithmique

Dans cette partie on implémente les classes Rule qui héritent de nos classes abstraites précedentes, c'est également là qu'on implémentera l'ensemble des fonctions (calcul de valuation, rank, unrank, count, ...)

In [ ]:
from functools import lru_cache

class UnionRule(ConstructorRule):
    def __init__(self, rule1, rule2):
        super().__init__((rule1, rule2))
        
    def _calc_valuation(self):
        rule1, rule2 = self.rules
        valuation = min(rule1.valuation, rule2.valuation)
        self._valuation = valuation
        
    @lru_cache(maxsize=None)
    def count(self, n):
        rule1, rule2 = self.rules
        return rule1.count(n) + rule2.count(n)
    
    @lru_cache(maxsize=None)
    def list(self, n):
        rule1, rule2 = self.rules
        return rule1.list(n) + rule2.list(n)
    
    @lru_cache(maxsize=None)
    def unrank(self, n, rank):
        rule1, rule2 = self.rules
        rule1count = rule1.count(n)
        
        if rank >= rule1count:
            return rule2.unrank(n, rank - rule1count)
        else:
            return rule1.unrank(n, rank)
            

        
class ProductRule(ConstructorRule):
    def __init__(self, rule1, rule2, cons):
        super().__init__((rule1, rule2))
        self._cons = cons
        
    def _calc_valuation(self):
        rule1, rule2 = self.rules
        valuation = rule1.valuation + rule2.valuation
        self._valuation = valuation
    
    @lru_cache(maxsize=None)
    def count(self, n):
        rule1, rule2 = self.rules
        countsum=0
        for i in range(n + 1):
            k=n-i
            if rule1.valuation > i or rule2.valuation > k:
                continue
            countsum += rule1.count(i) * rule2.count(k)
        return countsum
    
    @lru_cache(maxsize=None)
    def list(self, n):
        rule1, rule2 = self.rules
        res=[]
        
        for i in range(n+1):
            k=n-i
            if rule1.valuation > i or rule2.valuation > k:
                continue
            for el1 in rule1.list(i):
                for el2 in rule2.list(k):
                    res.append(self._cons(el1,el2))
        return res
    
    
    @lru_cache(maxsize=None)
    def unrank(self, n, rank):
        rule1, rule2 = self.rules 
        count, oldcount, i = 0, 0, 0
        while count <= rank and i <= n:
            k = n - i
            oldcount = count
            count += rule1.count(i) * rule2.count(k)
            i += 1
            
        if count <= rank:
            raise ValueError("Error called unrank with a too big rank")
            
        i -= 1
        j = rank - oldcount
        k = rule2.count(n-i)
        q, r = j // k, j % k
        
        return self._cons(rule1.unrank(i, q), rule2.unrank(n-i, r))
    

        
    
class EpsilonRule(ConstantRule):
    def __init__(self,obj):
        super().__init__(obj, 0)
        
    def count(self, n):
        return 1 if n==0 else 0
    
    def list(self, n):
        return [self._object] if n==0 else []
        

class SingletonRule(ConstantRule):
    def __init__(self,obj):
        super().__init__(obj, 1)
        
    def count(self, n):
        return 1 if n==1 else 0
    
    def list(self, n):
        return [self._object] if n==1 else []

In [ ]:
class BinaryTree():
    
    def __init__(self, children = None):
        """
        A binary tree is either a leaf or a node with two subtrees.
        
        INPUT:
            
            - children, either None (for a leaf), or a list of size excatly 2 
            of either two binary trees or 2 objects that can be made into binary trees
        """
        self._isleaf = (children is None)
        if not self._isleaf:
            if len(children) != 2:
                raise ValueError("A binary tree needs exactly two children")
            self._children = tuple(c if isinstance(c,BinaryTree) else BinaryTree(c) for c in children)
        self._size = None
        
    def __repr__(self):
        if self.is_leaf():
            return "leaf"
        return str(self._children)
    
    def __eq__(self, other):
        """
        Return true if other represents the same binary tree as self
        """
        if not isinstance(other, BinaryTree):
            return False
        if self.is_leaf():
            return other.is_leaf()
        return self.left() == other.left() and self.right() == other.right()
    
    
    def left(self):
        """
        Return the left subtree of self
        """
        return self._children[0]
    
    def right(self):
        """
        Return the right subtree of self
        """
        return self._children[1]
    
    def is_leaf(self):
        """
        Return true is self is a leaf
        """
        return self._isleaf
    
    
leaf = BinaryTree()

In [ ]:
def init_grammar(gram):
    
    for ruleName, ruleDef in gram.items():
        
        if not ruleDef._constant:
            for pr in ruleDef._parameters :
                if pr not in gram:
                    raise ValueError("A rule of the grammar ("+pr+") does not exist")
                
        ruleDef._set_grammar(gram)
        
    valuation = [rule.valuation for rule in gram.values()]
    valuation_old = [float('inf') for _ in valuation]

    while (valuation != valuation_old):
        valuation_old = valuation
        for k, v in gram.items():
            if not v._constant:
                v._calc_valuation()
        valuation = [rule.valuation for rule in gram.values()]
        
    if float('inf') in valuation:
        inf_rules = [ k for k, v in gram.items() if v.valuation == float('inf')]
        raise ValueError("The grammar is incorrect, there is a rule that yields no terminal : {}".format(inf_rules))
                
    print("Valuation = {}".format({k : v.valuation for k,v in gram.items()}))
        
        

In [ ]:
grammarTest = {
    "Axiom" : UnionRule("Vide","Axiom1"),
    "Axiom1" : ProductRule("SingA","IntermSingB",lambda a,b :"".join([a,b])),
    "Vide" : EpsilonRule(""),
    "SingA" : SingletonRule("A"),
    "SingB" : SingletonRule("B"),
    "IntermSingB" : ProductRule("Axiom","SingB",lambda a,b :"".join([a,b]))
}

In [ ]:
init_grammar(grammarTest)

In [ ]:
grammarTest["Axiom"].count(100)

In [ ]:
grammarTest["Axiom"].list(200)

In [ ]:
stringsum = lambda a,b : a + b
fiboGram = {    "Fib"    : UnionRule("Vide", "Cas1"),
                "Cas1"   : UnionRule("CasAu", "Cas2"),
                "Cas2"   : UnionRule("AtomB", "CasBAu"),
                "Vide"   : EpsilonRule(""),
                "CasAu"  : ProductRule("AtomA", "Fib", stringsum),
                "AtomA"  : SingletonRule("A"),
                "AtomB"  : SingletonRule("B"),
                "CasBAu" : ProductRule("AtomB", "CasAu", stringsum)}
init_grammar(fiboGram)

In [ ]:
fiboGram["Fib"].list(10)

In [ ]:
fiboGram["Fib"].count(100)

In [ ]:
fiboGram["Fib"].unrank(10,10)

In [ ]:
fiboGram["Fib"].unrank(10,fiboGram["Fib"].count(10))

In [ ]:
treeGram = {
    "Tree": UnionRule("Node", "Leaf"),
    "Node": ProductRule("Tree", "Tree", lambda a, b: BinaryTree([a, b])),
    "Leaf": SingletonRule(leaf)
}

In [ ]:
init_grammar(treeGram)

In [ ]:
treeGram["Tree"].list(5)